In [1]:
%pip install -e ../

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///C:/Users/elba_ro/Documents/projects/public_cag/corpus-annotation-graph-builder
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for cag (pyproject.toml): started
  Building editable for cag (pyproject.toml): finished with status 'done'
  Created wheel for cag: filename=cag-1.1.5-py3-none-any.whl size=3840 sha256=c0945cda034a14d1587c08a026a44f927a8723a6585643b92d6d8d470e735d3c
  Stored in directory: C:\Users\elba_ro\AppData\Local\Temp

In [2]:
%pip show cag

Name: cag
Version: 1.1.5
Summary: This is a general framework to create arango db graphs and annotate them.
Home-page: 
Author: 
Author-email: Roxanne El Baff <roxanne.elbaff@dlr.de>, Tobias Hecking <tobias.hecking@dlr.de>
License: 
Location: c:\users\elba_ro\appdata\roaming\python\python38\site-packages
Requires: dataclasses, empath, networkx, nltk, pyArango, pytest, python-arango, pyvis, spacy, spacy_arguing_lexicon, tomli, tqdm
Required-by: insightsnet-graphs
Note: you may need to restart the kernel to use updated packages.


In [3]:
from cag.framework.annotator.pipeline.pipeline_base import Pipeline
from cag.utils.config import Config
from tqdm import tqdm
from cag.graph_elements.base_graph import BaseGraph
from pyArango.graph import  EdgeDefinition
import cag


In [4]:
# make sure arangodb is up and running - Enter your credentials below
my_config = Config(
            url= "http://127.0.0.1:8529",
            user= "root",
            password= "oiE9O6s14rLwJ2SB",
            database= "_system",
            graph= "MyFirstCagGraph"
        )


## Create your first CAG Annotation Pipeline

In [5]:
# Extends the CAG Pipeline and implements the two methds: process_input and init_and_run
class MyFirstPipeline(Pipeline):
    def __init__(self, database_config: Config):
        super().__init__(database_config)
        self.period = None
    

    # Code for preprocessing the input data before annotating. In this case here, we convert the nodes into a list of tuples. 
    # Each tuple contains the text of the node and the _key (the unique id of the node)
    def process_input(self) -> list:
        processed = []
        for txt_node in tqdm(self.input):
            processed.append((txt_node.text, {"_key": txt_node._key}))

        return processed
    
    
    def init_and_run(self) -> list:

        # set the pipeline by adding pre defined pipelines from CAG or using cunstomized ones


        self.set_spacy_language_model("en_core_web_sm") #This is needed just because we are using NER spacy, otherwise, skip this if you are not using spacy pretrained pipes

        # Add Pipes
        ## The attributes of each pipe is defined in the cag.framework.annotator.registered_pipes
        self.add_annotation_pipe(name = "tok2vec", save_output= False,is_spacy=True, is_native=True) # mandatory for NER
        self.add_annotation_pipe(name="NamedEntityPipeOrchestrator", save_output=True, is_spacy=True) # spacy 
        self.add_annotation_pipe(name="EmpathPipeOrchestrator", save_output=True, is_spacy=True)
        self.init_pipe_stack()

        
            

## Initialize your first cag Pipeline

In [6]:
cag_pipeline = MyFirstPipeline(my_config)
cag_pipeline.spacy_n_processors= 2 # In case you are using spacy pipe, this flag can be set to enable multiprocessing,
# NOTE: If you are using spacy with transformer based feature, set the flag to 1 or else the pipeline will freeze (this is a spacy bug and not related to cag)
cag_pipeline.init_and_run()

cag          INFO     adding pipe with name NamedEntityPipeOrchestrator
cag          INFO     saving relation HasAnnotation to NamedEntityNode and from TextNode
cag          INFO     orchestrator is validated: True
cag          INFO     adding pipe with code ner
cag          INFO     adding pipe with name EmpathPipeOrchestrator
cag          INFO     saving relation HasAnnotation to EmpathNode and from TextNode
cag          INFO     orchestrator is validated: True
cag          INFO     adding pipe with code empath_component
cag          INFO     Defining pipe default and spacy stacks
cag          INFO     Pipes are ['tok2vec', 'ner', 'empath_component']


## Fetch the TextNode and Annotate & Save

In [7]:
#  Loop over your data, annotateA and save
coll = cag_pipeline.database_config.db["TextNode"]
docs = coll.fetchAll(limit=1000)
fetched = len(docs)
while docs != None and len(docs) >0:
    ## annotate

    # Set the INPUT - this will automatically call preprocess_input (make sure to implement it)
    cag_pipeline.reset_input_output()
    cag_pipeline.set_input(docs)
    
    cag_pipeline.annotate()
    cag_pipeline.save()
    
    cag_pipeline.reset_input_output()
    print(f"Processed {fetched} docs")
    docs = coll.fetchAll(limit=1000, skip=fetched)
    fetched = fetched + len(docs)

  0%|          | 0/3 [00:00<?, ?it/s]cag          INFO     saving annotations of Pipe(name='NamedEntityPipeOrchestrator', save_output=True, is_spacy=True, is_native=False, pipe_id_or_func='ner')
